In [1]:
from helpers import *
from lstmHelpers import *

import random
from random import randrange, shuffle
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
import time
from collections import deque
import pickle


filedir = "lstmData"

Using TensorFlow backend.


In [2]:
## GENERATE AND WRITE NEW STREAM DATA TO FILE

start = time.time()
total_time = time.time()

NUM_QUERY_FILES = 200  # If 0, use ALL classes; else, use n classes only
num_files_in_stream = 20 * NUM_QUERY_FILES
order_to_use = []

used_classes, unused_classes = load_classes("../", NUM_QUERY_FILES, display=False)

for i in range(NUM_QUERY_FILES):
    order_to_use.extend([i] * int(num_files_in_stream / NUM_QUERY_FILES))
    
shuffle(order_to_use)
generate_data_file("/trainingInfo_" + str(NUM_QUERY_FILES) + ".txt", filedir, used_classes, unused_classes, order_to_use, max_ramp_length=0.25)
shuffle(order_to_use)
generate_data_file("/testingInfo_" + str(NUM_QUERY_FILES) + ".txt", filedir, used_classes, unused_classes, order_to_use, max_ramp_length=0.25)

# code you want to evaluate
end = time.time()
print("Time elapsed: " + str(end-start) + " seconds, or " + str((end-start)/60.0) + " minutes")

Using classes...
Exact Classes: [9207, 8719, 4693, 9533, 2412, 5581, 7873, 1249, 8011, 3957, 4213, 2836, 2143, 8238, 1587, 4296, 242, 542, 3768, 4738, 8366, 4677, 578, 2513, 4817, 3512, 3971, 8320, 8426, 4220, 5402, 2576, 9164, 855, 8881, 2651, 1087, 8983, 1339, 7978, 6122, 4703, 7968, 4651, 1895, 2149, 4566, 2686, 7297, 1622, 2933, 1401, 2902, 2292, 359, 6230, 6761, 410, 5056, 2881, 2412, 9272, 7282, 1191, 9403, 755, 2531, 8661, 4756, 4977, 2325, 3907, 59, 3586, 7289, 8111, 9688, 1338, 6974, 9348, 5947, 7797, 2934, 7397, 9324, 5810, 9058, 7493, 6727, 7920, 485, 6369, 9722, 1928, 9728, 411, 6697, 8169, 9262, 203, 3297, 5891, 2777, 8299, 8942, 656, 5285, 697, 798, 4706, 6120, 8161, 976, 7537, 1678, 2978, 9247, 4230, 7611, 2864, 3684, 1927, 166, 4807, 2189, 3103, 7646, 9863, 5441, 6072, 3016, 7107, 3525, 2652, 6619, 9567, 1240, 1792, 1483, 9298, 1165, 1617, 1436, 5023, 47, 336, 7076, 3062]
Matched Classes: [3110, 4069, 1153, 589, 251, 2876, 88, 1813, 2954, 2759, 3745, 2842, 3874, 2226, 6

In [3]:
## READ AND GENERATE STREAM DATA FROM FILE
    
start = time.time()

audio_matrix = load_stream(name="/trainingInfo_" + str(NUM_QUERY_FILES) + ".txt", filedir=filedir)
composite_signal, composite_matches, preloaded_data = generate_composite_stream(audio_matrix)
#audio_matrix_test = load_stream(name="/testingInfo_" + str(NUM_QUERY_FILES) + ".txt", filedir=filedir)
#composite_signal_test, composite_matches_test, x = generate_composite_stream(audio_matrix_test, data_dict=preloaded_data)

audio_matrix = []
audio_matrix_test = []
preloaded_data = {}

NUM_CLASSES_USED = len(set(composite_matches))

end = time.time()
print("\nTime elapsed: " + str(end-start) + " seconds, or " + str((end-start)/60.0) + " minutes")



1%     2%     3%     4%     5%     6%     7%     8%     9%     10%     11%     12%     13%     14%     15%     16%     17%     18%     19%     20%     21%     22%     23%     24%     25%     26%     27%     28%     29%     30%     31%     32%     33%     34%     35%     36%     37%     38%     39%     40%     41%     42%     43%     44%     45%     46%     47%     48%     49%     50%     51%     52%     53%     54%     55%     56%     57%     58%     59%     60%     61%     62%     63%     64%     65%     66%     67%     68%     69%     70%     71%     72%     73%     74%     75%     76%     77%     78%     79%     80%     81%     82%     83%     84%     85%     86%     87%     88%     89%     90%     91%     92%     93%     94%     95%     96%     97%     98%     99%     100%     
Time elapsed: 82.59444618225098 seconds, or 1.3765741030375163 minutes


In [ ]:
## MINI-BATCH TESTING CELL

start = time.time()
n_mels = 128
seq_length = 10

start_point = 0
batch_length = 5000  # In # of sequences
audio_window= 64
batch_window = batch_length * audio_window

lstm_out = 350
batch_size = 32
dropout = 0.1
dropout_r = 0.1
number_outputs = NUM_CLASSES_USED

model = Sequential()
model.add(LSTM(lstm_out, input_shape=(seq_length, n_mels), dropout = dropout, recurrent_dropout = dropout_r))
model.add(Dense(number_outputs,activation='softmax'))  #softmax
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])  # sparse_categorical_crossentropy
print(model.summary())

composite_signal_length = len(composite_signal)

acc_0 = []
acc_1 = []
acc_2 = []
acc_3 = []

while start_point + batch_window < composite_signal_length:
    print("Training Data: " + str(start_point) + " / " + str(composite_signal_length) + " ~= " + str(round(start_point / composite_signal_length * 100)) + "%")
    training_data, training_classes = batch(composite_signal[start_point:start_point+batch_window], composite_matches[start_point:start_point+batch_window])
    training_data = np.array(training_data)
    history = model.fit(training_data, training_classes, epochs=4, batch_size=batch_size, verbose=0, shuffle=True)
    accuracies = history.history['acc']
    acc_0.append(accuracies[0])
    acc_1.append(accuracies[1])
    acc_2.append(accuracies[2])
    acc_3.append(accuracies[3])
    os.write(1, str(str(round(start_point / composite_signal_length * 100)) + "\n").encode())
    start_point += batch_window
    
os.write(1,"Completed Training\n".encode())
end = time.time()
print("Time elapsed: " + str(end-start) + " seconds, or " + str((end-start)/60.0) + " minutes")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 350)               670600    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               70200     
Total params: 740,800
Trainable params: 740,800
Non-trainable params: 0
_________________________________________________________________
None
Training Data: 0 / 105161216 ~= 0%
Training Data: 256000 / 105161216 ~= 0%
Training Data: 512000 / 105161216 ~= 0%
Training Data: 768000 / 105161216 ~= 1%
Training Data: 1024000 / 105161216 ~= 1%
Training Data: 1280000 / 105161216 ~= 1%
Training Data: 1536000 / 105161216 ~= 1%


In [ ]:
from matplotlib import pyplot as mp

plt.figure(figsize = (15,11))
plt.plot( np.arange(len(acc_0)), acc_0, label='Epoch 1')
plt.plot( np.arange(len(acc_1)), acc_1, label='Epoch 2')
plt.plot( np.arange(len(acc_2)), acc_2, label='Epoch 3')
plt.plot( np.arange(len(acc_3)), acc_3, label='Epoch 4')
plt.legend()

mp.savefig(filedir + '/accuracies_' + str(NUM_QUERY_FILES) + '.png', bbox_inches='tight')

In [ ]:
model.save(filedir + "/LSTMModel_" + str(NUM_QUERY_FILES) + ".h5")

## TESTING NETWORK

print("\nTesting Data")
testing_data, testing_classes = batch(composite_signal_test, composite_matches_test)
testing_data = np.array(testing_data)

scores = model.evaluate(testing_data, testing_classes)
print("Accuracy: %.2f%%" % (scores[1]*100))